In [ ]:
# Завантаження бібліотек
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
import time

In [ ]:
np.random.seed(777)

In [ ]:
# Функція активації ReLU
def relu(x):
    return np.maximum(0, x)

In [ ]:
# Функція активації Softmax
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

In [ ]:
# Шар згортки
def convolution_layer(input_value, kernel, bias, stride=1, padding=0):
    # Визначення розмірів вхідного значення
    input_depth, input_height, input_width = input_value.shape
    # Визначення розмірів ядра
    kernel_depth, kernel_height, kernel_width = kernel.shape
    # Розрахунок висоти та ширини вихідного значення
    output_height = (input_height - kernel_height + 2 * padding) // stride + 1
    output_width = (input_width - kernel_width + 2 * padding) // stride + 1
    # Ініціалізація вихідного значення нулями
    output_value = np.zeros((kernel_depth, output_height, output_width))
    # Додавання паддінгу до вхідного значення
    padded_input = np.pad(input_value, ((0, 0), (padding, padding), (padding, padding)), mode='constant')

    # Перебір по всіх глибинах ядра
    for d in range(kernel_depth):
        # Перебір по висоті вхідного значення з урахуванням кроку
        for i in range(0, input_height - kernel_height + 1, stride):
            # Перебір по ширині вхідного значення з урахуванням кроку
            for j in range(0, input_width - kernel_width + 1, stride):
                # Обчислення значення на виході через згортку та додавання зміщення
                output_value[d, i // stride, j // stride] = np.sum(
                    padded_input[:, i:i + kernel_height, j:j + kernel_width] * kernel[d]) + bias[d]
    # Повернення вихідного значення
    return output_value

In [ ]:
# Шар максимального пулінгу
def max_pooling(input_value, pool_size=2, stride=2):
    # Визначення розмірів вхідного значення
    input_depth, input_height, input_width = input_value.shape
    # Розрахунок висоти та ширини вихідного значення
    output_height = (input_height - pool_size) // stride + 1
    output_width = (input_width - pool_size) // stride + 1
    # Ініціалізація вихідного значення нулями
    output_value = np.zeros((input_depth, output_height, output_width))

    # Перебір по всіх глибинах вхідного значення
    for d in range(input_depth):
        # Перебір по висоті вхідного значення з урахуванням кроку
        for i in range(0, input_height - pool_size + 1, stride):
            # Перебір по ширині вхідного значення з урахуванням кроку
            for j in range(0, input_width - pool_size + 1, stride):
                # Обчислення максимального значення в поточному підвікні
                output_value[d, i // stride, j // stride] = np.max(
                    input_value[d, i:i + pool_size, j:j + pool_size])
    # Повернення вихідного значення
    return output_value

In [ ]:
# Повнозв'язний шар
def fully_connected_layer(input_value, weights, bias):
    output_value = np.dot(input_value.flatten(), weights) + bias
    return output_value

In [ ]:
# Функція втрат - категорійна крос-ентропія
def cross_entropy_loss(predicted, target):
    return -np.sum(target * np.log(predicted))

In [ ]:
# Шар Dropout
def dropout_layer(input_value, dropout_rate):
    dropout_mask = np.random.rand(*input_value.shape) < (1 - dropout_rate)
    return input_value * dropout_mask

In [ ]:
def my_model():
    def forward_propagation(image, stride=2, pool_size=2):
        # Встановлення ймовірності dropout
        dropout_rate = 0.25

        # Пропуск через перший шар згортки та функцію активації ReLU
        conv1_output = relu(convolution_layer(image, kernel_conv1, bias_conv1))
        # Застосування Dropout layer
        conv1_output = dropout_layer(conv1_output, dropout_rate)
        # Застосування шару Max Pooling
        pool1_output = max_pooling(conv1_output)

        # Пропуск через другий шар згортки та функцію активації ReLU
        conv2_output = relu(convolution_layer(pool1_output, kernel_conv2, bias_conv2))
        # Застосування Dropout layer
        conv2_output = dropout_layer(conv2_output, dropout_rate)
        # Застосування шару Max Pooling з заданим розміром вікна та кроком
        pool2_output = max_pooling(conv2_output, pool_size=pool_size, stride=stride)

        # Пропуск через третій шар згортки та функцію активації ReLU
        conv3_output = relu(convolution_layer(pool2_output, kernel_conv3, bias_conv3))

        # Пропуск через повнозв'язний шар
        f6_output = fully_connected_layer(conv3_output, weights_f6, bias_f6)
        # Пропуск через шар softmax для отримання ймовірностей класів
        output_value = softmax(fully_connected_layer(f6_output, weights_output, bias_output).reshape(1, -1))

        return conv1_output, pool1_output, conv2_output, pool2_output, conv3_output, f6_output, output_value

    def backward_propagation(image, target, learning_rate, stride=2, pool_size=2):
        # Використання глобальних змінних для параметрів моделі
        global kernel_conv1, bias_conv1, kernel_conv2, bias_conv2, kernel_conv3, bias_conv3, weights_f6, bias_f6, weights_output, bias_output

        # Виконання прямого поширення
        conv1_output, pool1_output, conv2_output, pool2_output, conv3_output, f6_output, output_value = forward_propagation(image, stride, pool_size)

        # Обчислення похибки на виході (delta) як різниця між виходом і цільовим значенням
        delta_output = output_value - target
        # Обчислення градієнтів для ваг і зміщень вихідного шару
        grad_weights_output = np.outer(f6_output, delta_output)
        grad_bias_output = delta_output

        # Обчислення похибки для попереднього шару
        delta_f6 = np.dot(delta_output, weights_output.T)
        delta_conv3 = np.dot(delta_f6, weights_f6.T).reshape(conv3_output.shape)

        # Обчислення градієнтів для ваг і зміщень повнозв'язного шару
        grad_weights_f6 = np.outer(conv3_output.flatten(), delta_f6)
        grad_bias_f6 = delta_f6

        # Обчислення похибки для шару Max Pooling 2
        delta_pool2 = np.zeros_like(pool2_output)
        for d in range(pool2_output.shape[0]):
            for i in range(0, pool2_output.shape[1], stride):
                for j in range(0, pool2_output.shape[2], stride):
                    if (i // stride < delta_conv3.shape[1]) and (j // stride < delta_conv3.shape[2]):
                        window = pool2_output[d, i:i + pool_size, j:j + pool_size]
                        mask = window == np.max(window)
                        delta_pool2[d, i:i + pool_size, j:j + pool_size] = mask * delta_conv3[d, i // stride, j // stride]

        # Обчислення похибки для другого шару згортки
        delta_conv2 = np.zeros_like(conv2_output)
        for d in range(conv2_output.shape[0]):
            for i in range(conv2_output.shape[1]):
                for j in range(conv2_output.shape[2]):
                    if (i // stride < delta_pool2.shape[1]) and (j // stride < delta_pool2.shape[2]):
                        delta_conv2[d, i, j] = np.sum(delta_pool2[d, i:i + stride, j:j + stride])

        # Використання функції активації ReLU для похибки
        delta_conv2 *= (conv2_output > 0)

        # Обчислення градієнтів для третього шару згортки
        grad_kernel_conv3 = convolution_layer(pool2_output, delta_conv3, np.zeros_like(bias_conv3), stride=1, padding=0)
        grad_bias_conv3 = np.sum(delta_conv3, axis=(1, 2))

        # Обчислення похибки для першого шару Max Pooling
        delta_pool1 = np.zeros_like(pool1_output)
        for d in range(pool1_output.shape[0]):
            for i in range(0, pool1_output.shape[1], stride):
                for j in range(0, pool1_output.shape[2], stride):
                    if (i // stride < delta_conv2.shape[1]) and (j // stride < delta_conv2.shape[2]):
                        window = pool1_output[d, i:i + pool_size, j:j + pool_size]
                        mask = window == np.max(window)
                        delta_pool1[d, i:i + pool_size, j:j + pool_size] = mask * delta_conv2[d, i // stride, j // stride]

        # Обчислення похибки для першого шару згортки, використовуючи збільшення похибки
        delta_conv1 = np.kron(delta_pool1, np.ones((1, stride, stride)))

        # Використання функції активації ReLU для похибки
        delta_conv1 *= (conv1_output > 0)

        # Обчислення градієнтів для другого шару згортки
        grad_kernel_conv2 = convolution_layer(pool1_output, delta_conv2, np.zeros_like(bias_conv2), stride=1, padding=0)
        grad_bias_conv2 = np.sum(delta_conv2, axis=(1, 2))

        # Обчислення градієнтів для першого шару згортки
        grad_kernel_conv1 = convolution_layer(image, delta_conv1, np.zeros_like(bias_conv1), stride=1, padding=0)
        grad_bias_conv1 = np.sum(delta_conv1, axis=(1, 2))

        # Перетворення градієнтів зміщень у вектор
        grad_bias_f6 = grad_bias_f6.reshape(-1)
        grad_bias_output = grad_bias_output.reshape(-1)

        # Оновлення параметрів моделі за допомогою градієнтів та швидкості навчання
        kernel_conv1 -= learning_rate * grad_kernel_conv1
        bias_conv1 -= learning_rate * grad_bias_conv1
        kernel_conv2 -= learning_rate * grad_kernel_conv2
        bias_conv2 -= learning_rate * grad_bias_conv2
        kernel_conv3 -= learning_rate * grad_kernel_conv3
        bias_conv3 -= learning_rate * grad_bias_conv3
        weights_f6 -= learning_rate * grad_weights_f6
        bias_f6 -= learning_rate * grad_bias_f6
        weights_output -= learning_rate * grad_weights_output
        bias_output -= learning_rate * grad_bias_output

    return forward_propagation, backward_propagation

In [ ]:
# Функція для ініціалізації вагів за методом Ксав'є
def initialize_weights(shape):
    stddev = np.sqrt(2.0 / np.prod(shape[:-1]))
    return np.random.normal(scale=stddev, size=shape)

# Функція для ініціалізації біасу нулями
def initialize_bias(shape):
    return np.zeros(shape)

# Кількість класів
num_classes = 10
# Ініціалізація вагових коефіцієнтів і зміщень для першого шару згортки
kernel_conv1 = initialize_weights((6, 5, 5))
bias_conv1 = initialize_bias(6)
# Ініціалізація вагових коефіцієнтів і зміщень для другого шару згортки
kernel_conv2 = initialize_weights((16, 5, 5))
bias_conv2 = initialize_bias(16)
# Ініціалізація вагових коефіцієнтів і зміщень для третього шару згортки
kernel_conv3 = initialize_weights((120, 5, 5))
bias_conv3 = initialize_bias(120)
# Ініціалізація вагових коефіцієнтів і зміщень для повнозв'язного шару
weights_f6 = initialize_weights((120, 84))
bias_f6 = initialize_bias(84)
# Ініціалізація вагових коефіцієнтів і зміщень для вихідного шару
weights_output = initialize_weights((84, num_classes))
bias_output = initialize_bias(num_classes)

In [ ]:
# Завантаження набору даних MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Перетворення типу даних зображень на float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Нормалізація зображень шляхом ділення на 255
X_train /= 255
X_test /= 255

# Зміна розміру зображень та додавання додаткового виміру для каналу
X_train_resize = np.expand_dims(np.pad(X_train, ((0,0), (2,2), (2,2)), mode='constant'), axis=1)
X_test_resize = np.expand_dims(np.pad(X_test, ((0,0), (2,2), (2,2)), mode='constant'), axis=1)

# Перетворення міток на one-hot вектори
Y_train = to_categorical(y_train, num_classes)
Y_test = to_categorical(y_test, num_classes)

# Вибір підмножини даних для тренування
subset_fraction = 0.02  # Фракція підмножини
subset_size = int(len(X_train) * subset_fraction)  # Розмір підмножини
subset_indices = np.random.choice(len(X_train), size=subset_size, replace=False)  # Випадковий вибір індексів
X_train_subset = X_train_resize[subset_indices]  # Підмножина тренувальних зображень
Y_train_subset = Y_train[subset_indices]  # Підмножина тренувальних міток

# Вибір підмножини даних для тестування
subset_size = int(len(X_test) * subset_fraction)  # Розмір підмножини
subset_indices = np.random.choice(len(X_test), size=subset_size, replace=False)  # Випадковий вибір індексів
X_test_subset = X_test_resize[subset_indices]  # Підмножина тестових зображень
Y_test_subset = Y_test[subset_indices]  # Підмножина тестових міток

In [ ]:
# Ініціалізація функцій прямого та зворотного поширення
forward_propagation, backward_propagation = my_model()

# Кількість епох для тренування моделі
epochs = 1
# Швидкість навчання
learning_rate = 0.0001
# Загальний час початку тренування
total_start_time = time.time()

# Списки для збереження точності та втрат
accuracy_list = []
loss_list = []

# Цикл по епохам
for epoch in range(epochs):
    epoch_loss = 0  # Змінна для зберігання загальної втрати за епоху
    start_time = time.time()  # Час початку епохи

    # Цикл по всіх зображеннях тренувальної підмножини
    for i in range(len(X_train_subset)):
        image = X_train_subset[i]  # Вибір зображення
        label = Y_train_subset[i]  # Вибір відповідної мітки
        # Виконання зворотного поширення для оновлення ваг
        backward_propagation(image, label, learning_rate)

    end_time = time.time()  # Час завершення епохи
    hours, remainder = divmod(end_time - start_time, 3600)
    minutes, seconds = divmod(remainder, 60)

    predicted_labels = []  # Список для зберігання передбачених міток

    # Цикл по всіх зображеннях тестової підмножини
    for i in range(len(X_test_subset)):
        image = X_test_subset[i]  # Вибір зображення
        label = Y_test_subset[i]  # Вибір відповідної мітки
        # Отримання виходу моделі
        output = forward_propagation(image)[6]
        # Обчислення втрати з використанням функції cross_entropy_loss
        loss = cross_entropy_loss(output, label)
        epoch_loss += loss  # Додавання втрати до загальної втрати за епоху
        predicted_label = np.argmax(output)  # Отримання передбаченої мітки
        predicted_labels.append(predicted_label)  # Додавання передбаченої мітки до списку

    # Обчислення середньої втрати за епоху
    avg_epoch_loss = epoch_loss / len(X_test_subset)
    loss_list.append(avg_epoch_loss)  # Додавання середньої втрати до списку втрат
    predicted_labels = np.array(predicted_labels)
    true_labels = np.argmax(Y_test_subset, axis=1)
    # Обчислення точності
    accuracy = np.mean(predicted_labels == true_labels)
    accuracy_list.append(accuracy)  # Додавання точності до списку точностей
    print(f"Epoch: {epoch+1}/{epochs} - Time: {int(hours)}h:{int(minutes)}m:{int(seconds)}s - Accuracy: {accuracy} - Loss: {avg_epoch_loss}")

total_end_time = time.time()
t_hours, t_remainder = divmod(total_end_time - total_start_time, 3600)
t_minutes, t_seconds = divmod(t_remainder, 60)
print(f"Total time: {int(t_hours)}h:{int(t_minutes)}m:{int(t_seconds)}s")

Save model

In [ ]:
# Імпорт бібліотеки для зберігання словника параметрів
import pickle

In [ ]:
def save_model_parameters(parameters, filename):
    """
    Зберігає параметри моделі у файл.

    Аргументи:
    parameters - словник, який містить параметри моделі (ваги, зміщення)
    filename - ім'я файлу для збереження параметрів

    Використання:
    save_model_parameters(model_parameters, 'model_parameters.pkl')
    """
    # Відкриваємо файл для запису в двійковому режимі
    with open(filename, 'wb') as model_file:
        # Зберігаємо словник параметрів у файл за допомогою pickle
        pickle.dump(parameters, model_file)

In [ ]:
# Створення словника з параметрами моделі
# Ключі словника є іменами параметрів, а значеннями є відповідні матриці ваг і зміщень

parameters_to_save = {
    # Параметри першого шару згортки
    'kernel_conv1': kernel_conv1, # Вагові коефіцієнти першого шару згортки
    'bias_conv1': bias_conv1, # Зміщення першого шару згортки
    # Параметри другого шару згортки
    'kernel_conv2': kernel_conv2, # Вагові коефіцієнти другого шару згортки
    'bias_conv2': bias_conv2, # Зміщення другого шару згортки
    # Параметри третього шару згортки
    'kernel_conv3': kernel_conv3, # Вагові коефіцієнти третього шару згортки
    'bias_conv3': bias_conv3, # Зміщення третього шару згортки
    # Параметри повнозв'язного шару
    'weights_f6': weights_f6, # Вагові коефіцієнти повнозв'язного шару
    'bias_f6': bias_f6, # Зміщення повнозв'язного шару
    # Параметри вихідного шару
    'weights_output': weights_output, # Вагові коефіцієнти вихідного шару
    'bias_output': bias_output # Зміщення вихідного шару
}
#Збереження параметрів моделі у файл 'my_model.pkl'
save_model_parameters(parameters_to_save, 'my_model.pkl')

# Visualization

In [ ]:
# Імпорт бібліотеки для візуалізації
import matplotlib.pyplot as plt

In [ ]:
# Створення графіку точності на епохах
plt.figure(figsize=(8, 6))
plt.plot(np.arange(1, epochs+1), accuracy_list, marker='o', linestyle='-')
plt.title('Accuracy on Epoch') # Заголовок графіку
plt.xlabel('Epoch') # Підпис осі X
plt.ylabel('Accuracy') # Підпис осі Y
plt.grid(True) # Включення сітки на графіку
plt.show() # Відображення графіку

In [ ]:
# Створення графіку втрат на епохах
plt.figure(figsize=(8, 6))
plt.plot(np.arange(1, epochs+1), loss_list, marker='o', linestyle='-', color='red')
plt.title('Loss on Epoch') # Заголовок графіку
plt.xlabel('Epoch') # Підпис осі X
plt.ylabel('Loss') # Підпис осі Y
plt.grid(True) # Включення сітки на графіку
plt.show()